In [1]:
import os
import cv2 as cv
import tensorflow as tf
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt

In [14]:
folder_path = 'C:/Users/ponmu/OneDrive/Desktop/aiml_ps/New folder/Converted'
train_path = 'C:/Users/ponmu/OneDrive/Desktop/aiml_ps/New_Data'
ds_train_ = tf.keras.utils.image_dataset_from_directory(
    train_path,
    labels='inferred',
    label_mode='binary',
    image_size=[224, 224],
    interpolation='nearest',
    batch_size=64,
    shuffle=True
)

# Calculate the split sizes
total_batches = len(ds_train_)
train_split = int(0.90 * total_batches)  # 90% for training

# Create training and validation subsets
ds_train = ds_train_.take(train_split)  # First 90%
ds_valid = ds_train_.skip(train_split)  # Remaining 10%

# Optional: Prefetch for performance
ds_train = ds_train.prefetch(buffer_size=tf.data.AUTOTUNE)
ds_valid = ds_valid.prefetch(buffer_size=tf.data.AUTOTUNE)

# Data Pipeline
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
data_train = (
    ds_train
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

data_valid = (
    ds_valid
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)


Found 592 files belonging to 2 classes.


In [15]:
model_2 = tf.keras.Sequential()
# Convolutional layers
model_2.add(tf.keras.layers.Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=(224,224, 3)))
model_2.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model_2.add(tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu'))
model_2.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model_2.add(tf.keras.layers.BatchNormalization())

model_2.add(tf.keras.layers.Flatten())
model_2.add(tf.keras.layers.Dense(1024,activation='relu'))
model_2.add(tf.keras.layers.Dropout(0.5))
model_2.add(tf.keras.layers.Dense(512,activation='relu'))
model_2.add(tf.keras.layers.Droupout(0.3))
model_2.add(tf.keras.layers.BatchNormalization())
model_2.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),metrics=['accuracy'])


model_2.fit(data_train,validation_data=data_valid,epochs=9,verbose=1)




c:\Users\ponmu\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


AttributeError: module 'keras._tf_keras.keras.layers' has no attribute 'Droupout'

In [1]:
model_2.save('model_4.h5')


NameError: name 'model_2' is not defined

In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.models import load_model
import os
from PIL import Image, UnidentifiedImageError
import numpy as np

# Custom Dataset Class
class TestImageDataset:
    def __init__(self, image_paths, transform=None):  # Corrected __init__ method
        self.image_paths = [path for path in image_paths if self.is_valid_image(path)]
        self.transform = transform

    def is_valid_image(self, img_path):
        try:
            Image.open(img_path).convert("RGB")
            return True
        except UnidentifiedImageError:
            print(f"Skipping invalid image: {img_path}")
            return False

    def __len__(self):  # Corrected __len__ method
        return len(self.image_paths)

    def __getitem__(self, idx):  # Corrected __getitem__ method
        img_path = self.image_paths[idx]
        image = keras_image.load_img(img_path, target_size=(150, 150))  # Adjust target size as needed
        image_array = keras_image.img_to_array(image) / 255.0  # Normalize to [0, 1]
        
        if self.transform:
            image_array = self.transform(image_array)

        return image_array, os.path.basename(img_path)



folder_path='C:/Users/ponmu/OneDrive/Desktop/aiml_ps/Test_Images'
image_paths = [os.path.join(folder_path, fname) for fname in os.listdir(folder_path)]
#folder_path_2 = 'C:/Users/ponmu/OneDrive/Desktop/aiml_ps/Test_Images'
#image_paths_2 = [os.path.join(folder_path, fname) for fname in os.listdir(folder_path_2)]
# Load the test dataset
test_dataset = TestImageDataset(image_paths)
#test_dataset_2 = TestImageDataset(image_paths_2)
# Convert to TensorFlow Dataset
def generator():
    for idx in range(len(test_dataset)):
        yield test_dataset[idx]

test_tf_dataset = tf.data.Dataset.from_generator(
    generator,
    output_signature=(
        tf.TensorSpec(shape=(150, 150, 3), dtype=tf.float32),  # Adjust shape as needed
        tf.TensorSpec(shape=(), dtype=tf.string)
    )
).batch(32)

# Load your trained model
model_path = r'C:\Users\ponmu\OneDrive\Desktop\aiml_ps\the_saga_6.keras'
model = load_model(model_path)
# Evaluate the model and make predictions
predictions = []

for batch_images, filenames in test_tf_dataset:
    outputs = model(batch_images, training=False)
    for filename, output in zip(filenames.numpy(), outputs.numpy()):
        label = 1 if output >= 0.5 else 0  # Sigmoid output for binary classification
        label_str = 'Real' if label == 1 else 'AI'
        predictions.append((filename.decode("utf-8"), label_str))



In [ ]:
import csv

#processed_predictions = [("image_" + str(int(filename.split('_')[1].split('.')[0])), label) for filename, label in predictions]
#processed_predictions.sort()
processed_predictions = sorted(
    [("image_" + str(int(filename.split('_')[1].split('.')[0])), label) for filename, label in predictions],
    key=lambda x: int(x[0].split('_')[1])
)
# Save to CSV file
csv_path = 'saga_6.csv'  # Save path can be adjusted
with open(csv_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Id', 'Label'])  # Write header
    writer.writerows(processed_predictions)  # Write data

# Print CSV location
print(f"Predictions saved to: {csv_path}")

# Print predictions for confirmation
for prediction in processed_predictions:
    print(prediction)


Predictions saved to: saga_6.csv


'for prediction in processed_predictions:\n    print(prediction)'

In [10]:
import pandas as pd

# Example data (you would replace this with your actual dataframe)
data = prediction  # Add all your data here
df = pd.DataFrame(data, columns=['Index', 'Category'])

# Modify the 'Index' column to show 'image_i'
df['Index'] = df['Index'].apply(lambda x: f'image_{x}')

print(df)


ValueError: Shape of passed values is (2, 1), indices imply (2, 2)